---
## Installing the required PowerShell modules
Code cells behave in the same way your code would in other environments, so you need to remember about common coding practices such as variable initialization and module/library imports. For this notebook you only need to make sure to install the required PowerShell modules since those are not installed by default.

In [9]:
##Installs modules necessary to run notebook
Install-Module DnsClient-PS -Force -AcceptLicense


---
## Connect to your Azure Sentinel workspace
Once you have configured your notebook, now you can connect to your workspace.

> **Note**: <br>
- We changed the default foreground colors in case you are using the "Light" notebook UI theme, since the yellow output will be hard to see.  Feel free to modify.
<br>
<br>


In [ ]:
##Get your configuration file settings
$nbcontentpath = "config.json"
if(!(test-path $nbcontentpath)){
    write-host "INFO: Your configuration path ($nbcontentpath) could not be located."
    write-host "INFO: Attempting to build the file path explicitly.  If this continues to be a problem, run 'dir' within the cell to find the current working directory and update the `$nbcontentpath variable accordingly."    
    $username = read-host "Enter the user name used for the notebook file explorer (the name of the top level folder):"
    $nbcontentpath = "users\$username\config.json"
}

##Path fix in case you picked up the cookie cutter configuration file (if you cloned repo from GitHub in terminal)
if(test-path $nbcontentpath){
    $content = gc $nbcontentpath | ?{$_ -match "cookiecutter"}
    if($content.Length -gt 0) {
        $nbcontentpath = "..\" + $nbcontentpath
    }    
}

try {
    $nbconfigcontent = Get-Content $nbcontentpath -ErrorAction Stop    
}
catch {
    write-host "ERROR: Your configuration path ($nbcontentpath) could not be located. Please fix before continuing further."    
}

##Set variables you will use throughout the notebook
$tenantId =  ($nbconfigcontent | ConvertFrom-Json).tenant_id
$subscriptionId = ($nbconfigcontent | ConvertFrom-Json).subscription_id
$resourceGroup = ($nbconfigcontent | ConvertFrom-Json).resource_group
$workspaceName = ($nbconfigcontent | ConvertFrom-Json).workspace_name
$workspaceId = ($nbconfigcontent | ConvertFrom-Json).workspace_id

Write-Host "SubscriptionId: " $subscriptionId 
Write-Host "TenantId: " $tenantId
Write-Host "WorkspaceId: " $workspaceId
Write-Host "workspaceName: " $workspaceName 

#Change the default colors used for PowerShell warnings as they make the Connect-AzAccount output difficult to see 
$Host.PrivateData.WarningBackgroundColor = "White"
$Host.PrivateData.WarningForegroundColor = "Black"

##Connect to selected subscription
Connect-AzAccount -UseDeviceAuthentication
Select-AzSubscription -SubscriptionId $subscriptionId -TenantId $tenantId

##Configure the Log Analytics workspace
$workspace = $null
$workspaces = Get-AzOperationalInsightsWorkspace -ResourceGroupName $resourceGroup
if($workspaces.Length -gt 1) {
    Write-Host "INFO: Multiple workspaces detected." 
    foreach($wksp in $workspaces){
        if($wksp.Name -eq $workspaceName)    {
          $workspace = $wksp
        }        
    }    
}
else {
     $workspace = $workspaces 
}
Write-Host "INFO: Ensure that the workspace -- {"$workspace.Name"} is the intended target workspace before continuing to the next cell."   
$workspace


---
## DNS Lookup Enrichment example

In [ ]:
##Query the Log Analytics table
$query = @"
    let blobsuffix = ".blob.core.windows.net";
    let filesuffix = ".file.core.windows.net";
    let queuesuffix = ".queue.core.windows.net";
    let tablesuffix = ".table.core.windows.net";
    AzureActivity
    | where TimeGenerated >= ago(30d)
    | where ResourceProviderValue == "Microsoft.Storage"
    | extend blobfqdn = strcat(Resource,blobsuffix), filefqdn = strcat(Resource,filesuffix)
    | extend queuefqdn = strcat(Resource,queuesuffix), tablefqdn = strcat(Resource,tablesuffix)
    | summarize by Resource,Caller,CallerIpAddress,blobfqdn    
"@

##Run query and add results to object. Now you can use object to display data or graph
$queryResults = Invoke-AzOperationalInsightsQuery -Workspace $Workspace -query $query
$resulttable = $queryResults.Results

# Build output file
$datetime = Get-Date -Format "yyyyMMddhhmmss"
$csvpath = "./outputs/" + $datetime + "results.csv"
"Resource,Caller,CallerIpAddress,blobfqdn,dnsresolveip" | Out-File $csvpath -Append -Force

foreach($fqdnresult in $resulttable) {

    $dnsresult = ((Resolve-Dns $fqdnresult.blobfqdn).Answers | Where-Object {$_.RecordType -eq "A"}).Address.IPAddressToString
    $stringinsert = "$($fqdnresult.Resource),$($fqdnresult.Caller),$($fqdnresult.CallerIpAddress),$($fqdnresult.blobfqdn),$($dnsresult)"
    $stringinsert | Out-File $csvpath -Append -Force
}

In [ ]:
$results = Import-Csv $csvpath
$results = $results | Select blobfqdn, dnsresolveip
$results | FT -AutoSize